In [116]:
#from identification import get_starting_points, get_vessel_graph, get_data_graph, identify_segments, get_segment_mask
from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer

In [141]:
from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer

In [177]:
import numpy as np
from matplotlib import pyplot as plt
import time

from skimage.morphology import skeletonize_3d, binary_dilation, convex_hull_image, ball, area_closing
from skimage import measure, segmentation, feature, filters, morphology
from scipy.ndimage import distance_transform_edt
from scipy.signal import fftconvolve
from scipy.ndimage import zoom
from mpl_toolkits.mplot3d import Axes3D
import pickle

In [178]:
# """This module contains functions necessary for the identification of vessel branches."""
# pylint: disable=import-error


from queue import SimpleQueue

import numpy as np
import networkx as nx

from scipy.ndimage import distance_transform_edt
from scipy.spatial.distance import euclidean
from skimage.draw import circle_perimeter, disk
from skimage.feature import peak_local_max


PROJECTION_ANGLES = {
    (45, 0): ["5", "6", "13", "9", "12a"],
    (-45, 0): ["5", "6", "13", "9", "12a"],
    (45, 20): ["5", "15"],
    (45, 20): ["5", "13"],
    (30, -30): ["5", "6", "13", "9", "12a"],
    (-30, -30): ["5", "13", "12a", "12b"],
    (-30, 30): ["5", "6", "9", "10", "10"],
}


def get_starting_points(img):
    """Get starting points for centerline detection algorithm. The starting point is selected
    as the maximum value of the euclidean distance transform (EDT) image of the segmented vessel.

    Args:
        img (np.array): EDT image of the segmented vessel.

    Returns:
        tuple: Starting point for centerline detection algorithm.
    """
    # initialize helper image
    helper = np.zeros_like(img)
    # find local maxima on EDT image.
    starting_points = peak_local_max(img)
    # get coordinates of local maxima
    helper[tuple(starting_points.T)] = 1
    starting_points = list(zip(*np.nonzero(helper)))
    # sort starting points by vessel diameter.
    starting_points.sort(key=lambda x: img[x])
    return starting_points


def get_sphere_points(center, radius, im_size):
    size = np.min([np.max(center) + 2*radius -1, im_size[0]-1, im_size[1]-1, im_size[2]-1])
    x, y, z = np.meshgrid(np.arange(size), np.arange(size), np.arange(size))
    d = np.sqrt((x-center[0])**2 + (y-center[1])**2 + (z-center[2])**2)
    surface_points = np.argwhere(np.isclose(d, radius))
    return [tuple(point) for point in surface_points]

def sphere_perimeter(coords, radius, shape):
    # Create arrays of indices for each dimension
    x_indices = np.arange(shape[0])
    y_indices = np.arange(shape[1])
    z_indices = np.arange(shape[2])
    
    # Create a 3D grid of indices for each dimension
    x_grid, y_grid, z_grid = np.meshgrid(x_indices, y_indices, z_indices, indexing='ij')
    
    # Calculate the distance of each point from the center
    distances = np.sqrt((x_grid - coords[0])**2 + (y_grid - coords[1])**2 + (z_grid - coords[2])**2)
    
    # Find the indices of points that are on the sphere surface
    sphere_indices = np.where(np.isclose(distances, radius))
    
    return sphere_indices

def sphere_full(coords, radius, shape):
    # Create arrays of indices for each dimension
    x_indices = np.arange(shape[0]-1)
    y_indices = np.arange(shape[1]-1)
    z_indices = np.arange(shape[2]-1)
    
    # Create a 3D grid of indices for each dimension
    x_grid, y_grid, z_grid = np.meshgrid(x_indices, y_indices, z_indices, indexing='ij')
    
    # Calculate the distance of each point from the center
    distances = np.sqrt((x_grid - coords[0])**2 + (y_grid - coords[1])**2 + (z_grid - coords[2])**2)
    
    sphere_indices = np.where(distances <= radius)
    
    # Create a binary mask with True values at the indices of points on the sphere surface
    mask = np.zeros(shape, dtype=bool)
    mask[sphere_indices] = True
    
    return mask

def is_inside_image(p, shape):
    return p[0] > 0 and p[1] > 0 and p[2] > 0 and p[0] > shape[0] and p[1] > shape[1] and p[2] > shape[2]

def get_points_of_interest(point, radius, visited, edt_img):
    """Get list of points that can be on the vessel centerline. The method utilizes local maxima
    to find the centerline on the basis of the EDT image of the vessel. The returned points are
    sorted by the EDT image value, with the pixels on the largest vessel at the beginning of the
    list.

    Args:
        point (tuple(int, int)): Point on centerline.
        radius (int): Radius of search circle.
        visited (np.array(bool)): Boolean array of visited image pixels.

    Returns:
        list: A list of points that can be on the vessel centerline.
    """
    # get points on circle circumference
    #circle_points = circle_perimeter(*point, radius=radius, method="andres")
    circle_points = sphere_perimeter(point, radius, edt_img.shape)
    #print("Circle points: ", len(circle_points), edt_img.shape)
    # create image with edt values of point on circle circumference
    helper = np.zeros_like(edt_img)
    helper[circle_points] = edt_img[circle_points]
    # find local maxima
    coords = peak_local_max(
        helper, min_distance=radius // 3, threshold_abs=2, threshold_rel=0.5
    )
    #print("Local peaks on surface: ", len(coords))
    # find coordinates of local maxima
    helper[circle_points] = 0
    helper[tuple(coords.T)] = 1
    helper[visited] = 0
    points_of_interest = list(zip(*np.nonzero(helper)))
    points_of_interest = list(filter(lambda x: is_inside_image(x, edt_img.shape), points_of_interest))
    # sort points by vessel diameter
    points_of_interest.sort(key=lambda x: edt_img[x], reverse=True)
    return points_of_interest


def get_disk_3d(center, radius):
    size = np.max(center) + 2*radius
    x, y, z = np.meshgrid(np.arange(size), np.arange(size), np.arange(size))
    d = np.sqrt((x-center[0])**2 + (y-center[1])**2 + (z-center[2])**2)
    surface_points = np.argwhere(d <= radius)
    return [tuple(point) for point in surface_points]

def generate_sphere(radius):
    large = ball(radius)
    small = ball(np.max(1, radius-1))
    padded_small = np.pad(small, 1, mode='constant')
    sphere = large - padded_small
    return sphere

def get_vessel_graph(seg_img):
    """Produce a graph which describes the vessel structure.

    Args:
        seg_img (np.array): Segmentation of vessel image.

    Returns:
        nx.Graph: Directed graph describing the vessel structure.
    """
    # perform euclidean distance transform on base image
    edt_img = distance_transform_edt(seg_img)
    print("EDT transform completed")
    # get starting points
    starting_point_stack = get_starting_points(edt_img)
    print("Finding starting points completed")

    # create vessel tree data structure
    vessel_graph = nx.DiGraph()
    # create array for holding information on visited pixels
    visited = np.zeros_like(edt_img).astype(bool)
    i = 0

    while starting_point_stack:
        i+=1
        # select new starting point and check if it was not visited already
        starting_point = starting_point_stack.pop()
        if visited[starting_point]:
            continue

        vessel_graph.add_node(starting_point)
        points_to_examine = [starting_point]
        if i % 10 == 0:
            print("Step i", i, len(starting_point_stack))
        j = 0
        while points_to_examine:
            j += 1
            if j % 100 == 0:
                print("Step: ", i, "-", j, " Points to examine: ", len(points_to_examine))
            # get point from queue
            point = points_to_examine.pop()
            # calculate vessel radius
            radius = int(edt_img[point])
            # if radius is too small, don't go further into this vessel
            # also limited by min_distance having to be >= 1 in peak_local_max
            if radius <= 1:
                continue
            # get points that can be on centreline
            points_of_interest = get_points_of_interest(point, radius, visited, edt_img)
                
            # eliminate points that are too close to each other
            for poi_1 in points_of_interest:
                for poi_2 in points_of_interest:
                    if poi_1 != poi_2 and euclidean(poi_1, poi_2) < np.maximum(edt_img[poi_1], edt_img[poi_2]):
                        if edt_img[poi_1] > edt_img[poi_2]:
                            points_of_interest.pop(points_of_interest.index(poi_1))
                        else:
                            points_of_interest.pop(points_of_interest.index(poi_2))

            # add points of interest to examination list and to vessel graph

            for poi in points_of_interest:
                # avoid node duplication
                if not (poi in points_to_examine or vessel_graph.has_node(poi)):
                    points_to_examine.append(poi)
                    vessel_graph.add_edge(point, poi)
                    
            # remove potential centreline pixels next to analyzed point to prevent going backwards
            disk_points = sphere_full(point, radius, visited.shape)

            visited[disk_points] = True

    return vessel_graph.to_undirected()


def remove_graph_components(graph):
    """Remove small (less than 4 nodes) graph components.

    Args:
        graph (nx.Graph): Undirected graph object from NetworkX library.

    Returns:
        nx.Graph: Graph without small components.
    """
    # get components
    components = list(nx.connected_components(graph))
    # remove small components
    for component in components:
        if len(component) < 2:
            for node in component:
                graph.remove_node(node)
    return graph


# pylint: disable=too-many-boolean-expressions,invalid-name
def connect_endings(graph, edt_img, multiplier=2.):
    """Fix discontinuities in vessel graph.

    Args:
        graph (nx.Graph): Graph object from NetworkX library.
        edt_img (np.array): Numpy array containing euclidean distance transformed image of vessel.
        multiplier (int, optional): Multiplier for maximum distance between nodes to connect.
        Defaults to 2.

    Returns:
        nx.Graph: Graph without discontinuities.
    """
    # find potential discontinuities
    endings = [node for node, degree in graph.degree() if degree == 1]
    # for every ending run BFS connection search
    while endings:
        # get point to find connection for
        start = endings.pop()
        # calculate search area
        r = edt_img[start] * multiplier
        search_area = int(r * r * np.pi)
        # setup BFS
        points = SimpleQueue()
        points.put(start)
        visited = []
        # run BFS on a restricted area
        while not points.empty() and search_area:
            search_area -= 1
            # get point
            x, y = points.get()
            visited.append((x, y))

            # check if point is a node and is a valid connection
            if graph.has_node((x, y)) and not nx.has_path(graph, start, (x, y)):
                graph.add_edge(start, (x, y))
                # this is to prevent accidentally creating bifurcations
                if (x, y) in endings:
                    endings.pop(endings.index((x, y)))
                break

            # add point to search if it is in segmentation mask and it is not visited
            for dx in range(-1, 2):
                for dy in range(-1, 2):
                    new_point = (x + dx, y + dy)
                    if (
                        x + dx >= 0
                        and x + dx < edt_img.shape[0]
                        and y + dy >= 0
                        and y + dy < edt_img.shape[1]
                        and new_point not in visited
                        and edt_img[x + dx, y + dy] > 0
                    ):
                        visited.append(new_point)
                        points.put(new_point)

    return graph

def connect_endings_mst(graph, edt_image, multiplier=10):
    nodes = list(graph.nodes())
    n = len(nodes)
    
    G = nx.Graph()
    G.add_nodes_from(nodes)

    for i in range(n):
        for j in range(i+1, n):
            d = euclidean(nodes[i], nodes[j]) 
            d_thresh = np.maximum(edt_image[nodes[i]], edt_image[nodes[j]]) * multiplier
            if d <= d_thresh:
                G.add_edge(nodes[i], nodes[j], weight=d)
    return nx.minimum_spanning_tree(G)


def sorted_nodes_by_distance(graph, node):
    distances = {}
    for n in list(graph.nodes):
        if n == node:
            continue
        dist = euclidean(node, n)
        distances[n] = dist
    sorted_nodes = sorted(distances, key=distances.get)
    return sorted_nodes
    
# pylint: disable=too-many-boolean-expressions,invalid-name
def connect_endings_3d(graph, edt_img, multiplier=3.):
    """Fix discontinuities in vessel graph.

    Args:
        graph (nx.Graph): Graph object from NetworkX library.
        edt_img (np.array): Numpy array containing euclidean distance transformed image of vessel.
        multiplier (int, optional): Multiplier for maximum distance between nodes to connect.
        Defaults to 2.

    Returns:
        nx.Graph: Graph without discontinuities.
    """
    # find potential discontinuities
    endings = [node for node, degree in graph.degree() if degree <= 1]
    root = np.unravel_index(np.argmax(edt_img), edt_img.shape)
    
    if root not in graph.nodes:
        graph.add_node(root)
        
    print("Endings: ", len(endings))
    # for every ending run BFS connection search
    while endings:
        # get point to find connection for
        start = endings.pop()
        # calculate search area
        r = edt_img[start] * multiplier
        search_area = int(4/3 * r * r * r * np.pi)
        # setup BFS
        points = SimpleQueue()
        points.put(start)
        visited = np.zeros_like(edt_img).astype(bool)

        # run BFS on a restricted area
        i = 0

        to_visit = sorted_nodes_by_distance(graph, start)
        print("Endings: ", len(endings))

        while i < len(to_visit):
            if nx.has_path(graph, root, start) or euclidean(start, to_visit[i]) > edt_img[start] * multiplier:
                break
            curr = to_visit[i]
            i += 1
            if not nx.has_path(graph, start, curr) and curr != start:
                graph.add_edge(start, curr)
                if curr in endings:
                    endings.pop(endings.index(curr))
                break

    return graph


def get_node_data(node, prev_node, next_node, edt_img):
    """Get data for node in vessel graph.

    Args:
        node (tuple): Current node for which data is extracted.
        prev_node (tuple): Previous node in graph traversal.
        next_node (tuple): Next node in graph traversal.
        edt_img (np.array): Euclidean distance transformed image of segmented vessel.

    Returns:
        dict: Dictionary containing node data.
    """
    data = {}
    # get vessel diameter
    data["vessel_diameter"] = float(edt_img[node])
    # get vessel diameter gradient
    data["vessel_diameter_grad"] = float(edt_img[node] - edt_img[prev_node])
    # get vessel fragment length
    v1 = np.subtract(prev_node, node).astype(np.float32)
    data["vessel_length"] = float(np.linalg.norm(v1))
    # get angle between nodes
    v1 /= np.linalg.norm(v1)
    v2 = np.subtract(next_node, node).astype(np.float32)
    v2 /= np.linalg.norm(v2)
    data["angle"] = float(np.arccos(np.clip(np.dot(v1, v2), -1.0, 1.0)))
    return data


def parametrize_graph(vessel_graph, edt_img):
    """Retrieve general segment data from detailed vessel graph.

    Args:
        vessel_graph (nx.Graph): Graph describing detailed vessel structure.
        edt_img (np.array): Numpy array containing euclidean distance transformed segmentation of
        vessel.

    Returns:
        nx.Digraph: Directed, parametrized graph containing vessel segment data in nodes.
    """
    # get nodes and sort them by vessel diameter
    # this list can be used in the future to include disconnected graph components
    nodes = [node for node, degree in vessel_graph.degree() if degree == 1]
    nodes.sort(key=lambda n: edt_img[n])
    # select starting point
    start = nodes.pop()
    # initiate graph
    data_graph = nx.Graph()
    # this variable is for identifying nodes in parametrized graph
    data_graph.add_node(start)
    # initiate paths to explore from this point
    paths_to_explore = [(start, neighbor) for neighbor in vessel_graph.neighbors(start)]
    visited = [start]
    # check every path for bifurcations or endings
    while paths_to_explore:
        # get relevant data
        start, node = paths_to_explore.pop()
        prev = start
        # flag node as visited
        visited.append(node)
        # create object to gather data from vessel segment
        data = {"nodes": {start: get_node_data(start, start, node, edt_img)}}
        # measure length of vessel segment
        total_len = 0
        vessel_diameters = []
        # traverse segment while no bifurcations or endings were detected
        while vessel_graph.degree(node) == 2:
            # get next node
            neighbors = list(vessel_graph.neighbors(node))
            neighbors.pop(neighbors.index(prev))
            next_node = neighbors[0]
            # gather data on next node
            data["nodes"][node] = get_node_data(node, prev, next_node, edt_img)
            total_len += data["nodes"][node]["vessel_length"]
            vessel_diameters.append(data["nodes"][node]["vessel_diameter"])
            # go ahead
            prev = node
            node = next_node
            visited.append(node)
        data["nodes"][node] = get_node_data(node, prev, node, edt_img)
        total_len += data["nodes"][node]["vessel_length"]
        vessel_diameters.append(data["nodes"][node]["vessel_diameter"])
        # this happens after ending/bifurcation was detected
        data["segment_length"] = total_len
        data["average_vessel_diameter"] = sum(vessel_diameters) / len(vessel_diameters)
        # add node with collected data
        data_graph.add_node(node)
        data_graph.add_edge(start, node, **data)
        # add paths to explore if there are new ones
        paths_to_explore.extend(
            [
                (node, neighbor)
                for neighbor in vessel_graph.neighbors(node)
                if neighbor not in visited
            ]
        )
    return data_graph


def choose_root_node(data_graph):
    """Select root node for data graph.

    Args:
        data_graph (nx.Graph): Undirected vessel data graph.

    Returns:
        tuple: coordinates of root node.
    """
    # select edge with biggest average vessel diameter
    edges = [
        k
        for k, _ in sorted(
            nx.get_edge_attributes(data_graph, "average_vessel_diameter").items(),
            key=lambda item: item[1],
        )
    ]
    n1, n2 = edges[-1]
    # select root by node degree
    if data_graph.degree(n1) == 1 and data_graph.degree(n2) != 1:
        return n1
    if data_graph.degree(n2) == 1 and data_graph.degree(n1) != 1:
        return n2
    # select root by biggest vessel diameter
    n1_diameter = data_graph[n1][n2]["nodes"][n1]["vessel_diameter"]
    n2_diameter = data_graph[n1][n2]["nodes"][n2]["vessel_diameter"]
    return n1 if n1_diameter > n2_diameter else n2


def clean_data_graph(data_graph, min_segment_length=20):
    """Clean data graph by removing small segments, nodes of degree 2 and selecting a new root.

    Args:
        data_graph (nx.Graph): Undirected vessel data graph.
        min_segment_length (int, optional): Minimum segment length permissible in graph.
        Defaults to 20[px].

    Returns:
        nx.Graph: Cleaned vessel data graph.
    """
    # remove small segments
    for edge, segment_length in nx.get_edge_attributes(
        data_graph, "segment_length"
    ).items():
        if segment_length < min_segment_length:
            data_graph.remove_edge(*edge)
    # remove isolated nodes (degree == 0)
    data_graph.remove_nodes_from(list(nx.isolates(data_graph)))
    # merge segments for nodes of degree 2
    while [n for n, degree in data_graph.degree() if degree == 2]:
        for node, degree in list(data_graph.degree()):
            if degree != 2:
                continue
            n1, n2 = list(data_graph.neighbors(node))
            new_edge_data = {}
            new_edge_data["nodes"] = (
                data_graph[node][n1]["nodes"] | data_graph[node][n2]["nodes"]
            )
            new_edge_data["segment_length"] = sum(
                node_data["vessel_length"]
                for node_data in new_edge_data["nodes"].values()
            )
            new_edge_data["average_vessel_diameter"] = sum(
                node_data["vessel_diameter"]
                for node_data in new_edge_data["nodes"].values()
            ) / len(new_edge_data["nodes"])
            data_graph.remove_node(node)
            data_graph.add_edge(n1, n2, **new_edge_data)
    # select new root
    root_node = choose_root_node(data_graph)
    nx.set_node_attributes(data_graph, False, "root")
    data_graph.nodes[root_node]["root"] = True
    return data_graph


def identify_segments(data_graph, primary_angle, secondary_angle):
    """Identify vessel segments in vessel data graph.

    Args:
        data_graph (nx.Graph): Undirected vessel data graph.
        primary_angle (int): Positioner primary angle.
        secondary_angle (int): Positioner secondary angle.

    Raises:
        AttributeError: if angles are outside of the expected bounds (at least 20 degrees off the
        expected values defined in PROJECTION_ANGLES).

    Returns:
        nx.Graph: vessel data graph with labeled vessel segments.
    """
    for angles, vessel_labels in PROJECTION_ANGLES.items():
        primary, secondary = angles
        if (
            abs(primary_angle - primary) <= 20
            and abs(secondary_angle - secondary) <= 20
        ):
            labels = vessel_labels.copy()
            break
    else:
        raise AttributeError("Image has incorrect projection angles.")

    for k, v in data_graph.nodes(data="root"):
        if v:
            root_node = k
            break

    current_label = ""
    for n1, n2 in nx.bfs_edges(data_graph, root_node):
        if labels:
            current_label = labels.pop(0)
        data_graph[n1][n2]["vessel_label"] = current_label

    return data_graph


def get_segment_mask(data_graph, seg_img):
    """Generate representation of vessel segments in the form of segmentation masks.

    Args:
        data_graph (nx.Graph): NetworkX graph containing general vessel information.
        seg_img (np.array): Numpy array containing segmented vessel image.

    Returns:
        tuple(np.array, dict): Numpy array containing segment mask and label dict.
    """
    masks = []
    labels = {}
    for idx, nodes in enumerate(data_graph.edges.data("nodes")):
        try:
            labels[idx + 1] = data_graph[nodes[0]][nodes[1]]["vessel_label"]
        except KeyError:
            labels[idx + 1] = "UNKN"
            # WARN print(idx,nodes[0],nodes[1]," no label")
        # calculate distances from nodes in segment
        mask = np.ones_like(seg_img)
        mask[nodes[:-1]] = 0
        idxs = tuple(np.array(list(nodes[-1].keys())).T)
        mask[idxs] = 0
        mask = distance_transform_edt(mask)
        # append to summary mask
        masks.append(mask)
    masks = np.stack(masks)
    # select label closest to point
    result = np.argmin(masks, axis=0) + 1
    result[seg_img == 0] = 0
    result = result.astype(np.uint8)
    return result, labels


def get_data_graph(seg_img):
    """Function collecting steps for creating a data graph.

    Args:
        seg_img (np.array): Numpy array containing segmented vessel image.

    Returns:
        nx.Graph: Vessel data graph.
    """
    edt_img = distance_transform_edt(seg_img)
    vessel_graph = get_vessel_graph(seg_img)
    print(nx.info(vessel_graph))
    vessel_graph = remove_graph_components(vessel_graph)
    print(nx.info(vessel_graph))
    vessel_graph = connect_endings_3d(vessel_graph, edt_img)
    print(nx.info(vessel_graph))
    data_graph = parametrize_graph(vessel_graph, edt_img)
    data_graph = clean_data_graph(data_graph)
    return data_graph


### Visualisation utilities

In [179]:
def visualize_addition(base, base_with_addition):
    base = (base.copy() > 0).astype(np.uint8)
    addition = (base_with_addition > 0).astype(np.uint8)
    addition[base == 1] = 0
    ColorMapVisualizer(base + addition * 4).visualize()
    
def visualize_lsd(lsd_mask):
    ColorMapVisualizer(lsd_mask.astype(np.uint8)).visualize()
    
def visualize_gradient(lsd_mask):
    ColorMapVisualizer(lsd_mask.astype(np.uint8)).visualize(gradient=True)
    
def visualize_mask_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8), binary=True).visualize()
    
def visualize_mask_non_bin(mask):
    VolumeVisualizer((mask > 0).astype(np.uint8) * 255, binary=False).visualize()
    
def visualize_skeleton(mask, visualize_mask=True, visualize_both_versions=False):
    skeleton = skeletonize_3d((mask > 0).astype(np.uint8))
    if not visualize_mask or visualize_both_versions:
        VolumeVisualizer(skeleton, binary=True).visualize()
    if visualize_mask or visualize_both_versions:
        skeleton = skeleton.astype(np.uint8) * 4
        mask = (mask > 0).astype(np.uint8) * 3
        mask[skeleton != 0] = 0
        ColorMapVisualizer(skeleton + mask).visualize()

def visualize_ultimate(lsd, base_mask):
    visualize_lsd(lsd)
    visualize_mask_non_bin(lsd)
    visualize_addition(base_mask, lsd)
    visualize_skeleton(lsd, visualize_mask=True)
    
def plot_3d_graph(G):
    pos = nx.spring_layout(G, dim=3)
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    for node in G.nodes():
        (x,y,z) = pos[node]
        ax.scatter(x, y, z, c='b', marker='o')
        ax.text(x, y, z, node, fontsize=10)
    for edge in G.edges():
        (u,v) = edge
        (x1,y1,z1) = pos[u]
        (x2,y2,z2) = pos[v]
        ax.plot([x1,x2], [y1,y2], [z1,z2], 'k-', alpha=0.5)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()
    
def draw_graph_on_model(binary_model, graph):
    mask = np.zeros(binary_model.shape, dtype=np.uint8)
    mask[binary_model] = 30
    

    for edge in graph.edges:
        node1, node2 = edge
        x1, y1, z1 = node1
        x2, y2, z2 = node2
        line_x = np.linspace(x1, x2, num=300, endpoint=True, dtype=np.int32)
        line_y = np.linspace(y1, y2, num=300, endpoint=True, dtype=np.int32)
        line_z = np.linspace(z1, z2, num=300, endpoint=True, dtype=np.int32)
        for i in range(len(line_x)):
            mask[line_x[i], line_y[i], line_z[i]] = 255

    for node in graph.nodes:
        x, y, z = node
        mask[x, y, z] = 40
#         for dx in (-1, 2):
#             for dy in (-1, 2):
#                 for dz in (-1, 2):
#                     mask[x+dx, y+dy, z+dz] = 150

    return mask


### Read data

In [244]:
volume = np.fromfile('../data/P13/data.raw', dtype=np.uint8)

In [245]:
volume = volume.reshape(877, 488, 1132)
volume = volume[200:700, :, 100:650]
val = 20

volume[:, 200:, -35:] = val
volume[-150:, 200:, -100:-35] = val
volume[:, -200:-150, -100:-35] = val
volume[:, :210, -50:] = val
volume[:150, :150, -85:-50] = val

volume = volume[-170:-40,200:480,30:260]
volume[-90:, :, -110:] = val
volume[:, -80:, -92:] = val

In [246]:
VolumeVisualizer(volume, binary=False).visualize() 

In [247]:
mask = volume > 32

In [248]:
def get_main_regions(binary_mask, min_size=10_000, connectivity=3):
    labeled = measure.label(binary_mask, connectivity=connectivity)
    region_props = measure.regionprops(labeled)
    
    main_regions = np.zeros(binary_mask.shape)
    bounding_boxes = []
    for props in region_props:
        if props.area >= min_size:
            bounding_boxes.append(props.bbox)
            main_regions = np.logical_or(main_regions, labeled==props.label)
            
    lower_bounds = np.min(bounding_boxes, axis=0)[:3]
    upper_bounds = np.max(bounding_boxes, axis=0)[3:]

    return main_regions[
        lower_bounds[0]:upper_bounds[0],
        lower_bounds[1]:upper_bounds[1],
        lower_bounds[2]:upper_bounds[2],
    ], bounding_boxes

In [249]:
main_regions, bounding_boxes = get_main_regions(mask, min_size=25_000)
print('number of main regions:', len(bounding_boxes))
mask_main = main_regions
# mask = None
# visualize_mask_non_bin(mask_main)

number of main regions: 1


In [250]:
visualize_mask_bin(mask_main)

In [251]:
#ColorMapVisualizer(img_edt.astype(np.uint8)).visualize()

In [252]:
#img_edt = distance_transform_edt(mask_main)

In [253]:
def spherical_kernel(outer_radius, thickness=1, filled=True):    
    outer_sphere = morphology.ball(radius=outer_radius)
    if filled:
        return outer_sphere
    
    thickness = min(thickness, outer_radius)
    
    inner_radius = outer_radius - thickness
    inner_sphere = morphology.ball(radius=inner_radius)
    
    begin = outer_radius - inner_radius
    end = begin + inner_sphere.shape[0]
    outer_sphere[begin:end, begin:end, begin:end] -= inner_sphere
    return outer_sphere

def convolve_with_ball(img, ball_radius, dtype=np.uint16, normalize=True, fft=True):
    kernel = spherical_kernel(ball_radius, filled=True)
    if fft:
        convolved = fftconvolve(img.astype(dtype), kernel.astype(dtype), mode='same')
    else:
        convolved = signal.convolve(img.astype(dtype), kernel.astype(dtype), mode='same')
    
    if not normalize:
        return convolved
    
    return (convolved / kernel.sum()).astype(np.float16)

def calculate_reconstruction(mask, kernel_sizes=[10, 9, 8, 7], fill_threshold=0.5, iters=1, conv_dtype=np.uint16, fft=True):
    kernel_sizes_maps = []
    mask = mask.astype(np.uint8)
    
    for i in range(iters):
        kernel_size_map = np.zeros(mask.shape, dtype=np.uint8)

        for kernel_size in kernel_sizes:
            fill_percentage = convolve_with_ball(mask, kernel_size, dtype=conv_dtype, normalize=True, fft=fft)
            
            above_threshold_fill_indices = fill_percentage > fill_threshold
            kernel_size_map[above_threshold_fill_indices] = kernel_size + 1

            mask[above_threshold_fill_indices] = 1
            
            print(f'Iteration {i + 1} kernel {kernel_size} done')

        kernel_sizes_maps.append(kernel_size_map)
        print(f'Iteration {i + 1} ended successfully')

    return kernel_sizes_maps

In [254]:
scaled_mask = zoom(mask_main, zoom=0.7, order=2)

In [255]:
s_recos = calculate_reconstruction(scaled_mask, 
                                   kernel_sizes=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
                                   iters=3)

Iteration 1 kernel 1 done
Iteration 1 kernel 2 done
Iteration 1 kernel 3 done
Iteration 1 kernel 4 done
Iteration 1 kernel 5 done
Iteration 1 kernel 6 done
Iteration 1 kernel 7 done
Iteration 1 kernel 8 done
Iteration 1 kernel 9 done
Iteration 1 kernel 10 done
Iteration 1 kernel 11 done
Iteration 1 kernel 12 done
Iteration 1 ended successfully
Iteration 2 kernel 1 done
Iteration 2 kernel 2 done
Iteration 2 kernel 3 done
Iteration 2 kernel 4 done
Iteration 2 kernel 5 done
Iteration 2 kernel 6 done
Iteration 2 kernel 7 done
Iteration 2 kernel 8 done
Iteration 2 kernel 9 done
Iteration 2 kernel 10 done
Iteration 2 kernel 11 done
Iteration 2 kernel 12 done
Iteration 2 ended successfully
Iteration 3 kernel 1 done
Iteration 3 kernel 2 done
Iteration 3 kernel 3 done
Iteration 3 kernel 4 done
Iteration 3 kernel 5 done
Iteration 3 kernel 6 done
Iteration 3 kernel 7 done
Iteration 3 kernel 8 done
Iteration 3 kernel 9 done
Iteration 3 kernel 10 done
Iteration 3 kernel 11 done
Iteration 3 kernel 1

In [256]:
bin_reco = s_recos[-1] > 0

In [257]:
%%time
edt_img = distance_transform_edt(bin_reco)

CPU times: total: 203 ms
Wall time: 206 ms


In [194]:
ColorMapVisualizer(edt_img.astype(np.uint8)).visualize()

In [258]:
%%time
vessel_graph = get_vessel_graph(bin_reco)
print(nx.info(vessel_graph))

EDT transform completed
Finding starting points completed
Step i 40 2753
Step i 60 2733
Step i 80 2713
Step i 140 2653
Step i 150 2643
Step i 160 2633
Step i 170 2623
Step i 190 2603
Step i 200 2593
Step i 230 2563
Step i 250 2543
Step i 270 2523
Step i 290 2503
Step i 320 2473
Step i 350 2443
Step i 370 2423
Step i 380 2413
Step i 390 2403
Step i 400 2393
Step i 410 2383
Step i 420 2373
Step i 430 2363
Step i 440 2353
Step i 450 2343
Step i 460 2333
Step i 470 2323
Step i 480 2313
Step i 490 2303
Step i 500 2293
Step i 510 2283
Step i 520 2273
Step i 530 2263
Step i 540 2253
Step i 550 2243
Step i 560 2233
Step i 570 2223
Step i 580 2213
Step i 590 2203
Step i 600 2193
Step i 610 2183
Step i 620 2173
Step i 630 2163
Step i 640 2153
Step i 650 2143
Step i 660 2133
Step i 670 2123
Step i 680 2113
Step i 690 2103
Step i 700 2093
Step i 710 2083
Step i 720 2073
Step i 730 2063
Step i 740 2053
Step i 750 2043
Step i 760 2033
Step i 770 2023
Step i 780 2013
Step i 790 2003
Step i 800 1993
S

<timed exec>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.



In [259]:
model_with_graph = draw_graph_on_model(bin_reco, vessel_graph)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [260]:
%%time
vessel_graph_rm = remove_graph_components(vessel_graph_cn)
print(nx.info(vessel_graph_rm))

Graph with 2554 nodes and 2220 edges
CPU times: total: 15.6 ms
Wall time: 9 ms


<timed exec>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.



In [261]:
model_with_graph = draw_graph_on_model(bin_reco, vessel_graph_rm)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [199]:
pickle.dump(vessel_graph_rm, open('removed-small.pickle', 'wb'))

In [262]:
# Orignal Witold method adjusted to 3D

def connect_endings_3d(graph, edt_img, multiplier=2.5):
    """Fix discontinuities in vessel graph.

    Args:
        graph (nx.Graph): Graph object from NetworkX library.
        edt_img (np.array): Numpy array containing euclidean distance transformed image of vessel.
        multiplier (int, optional): Multiplier for maximum distance between nodes to connect.
        Defaults to 2.

    Returns:
        nx.Graph: Graph without discontinuities.
    """
    # find potential discontinuities
    endings = [node for node, degree in graph.degree() if degree <= 1]
    
    print("Found endings: ", len(endings))
    # for every ending run BFS connection search
    i = 0
    while endings:
        # get point to find connection for
        start = endings.pop()
        # calculate search area
        r = edt_img[start] * multiplier
        search_area = int(4/3 * r * r * r * np.pi)
        # setup BFS
        points = SimpleQueue()
        points.put(start)
        visited = []
        # run BFS on a restricted area
        j = 0
        while not points.empty() and search_area:
            if j % 10 == 0:
                print(f"i: {i} j: {j} points: ", points.qsize())
            search_area -= 1
            # get point
            x, y, z = points.get()
            visited.append((x, y, z))

            # check if point is a node and is a valid connection
            if graph.has_node((x, y, z)) and not nx.has_path(graph, start, (x, y, z)):
                graph.add_edge(start, (x, y, z))
                # this is to prevent accidentally creating bifurcations
                if (x, y, z) in endings:
                    endings.pop(endings.index((x, y, z)))
                break

            # add point to search if it is in segmentation mask and it is not visited
            for dx in range(-1, 2):
                for dy in range(-1, 2):
                    for dz in range(-1, 2):
                        new_point = (x + dx, y + dy, z + dz)
                        if (
                            x + dx >= 0
                            and x + dx < edt_img.shape[0]
                            and y + dy >= 0
                            and y + dy < edt_img.shape[1]
                            and z + dz >= 0
                            and z + dz < edt_img.shape[2]
                            and new_point not in visited
                            and edt_img[x + dx, y + dy, z + dz] > 0
                        ):
                            visited.append(new_point)
                            points.put(new_point)
            j += 1
        i += 1

    return graph

In [263]:
vessel_graph_cn = connect_endings_3d(vessel_graph, edt_img, 3)

Found endings:  2571
i: 0 j: 0 points:  1
i: 1 j: 0 points:  1
i: 2 j: 0 points:  1
i: 3 j: 0 points:  1
i: 4 j: 0 points:  1
i: 5 j: 0 points:  1
i: 6 j: 0 points:  1
i: 7 j: 0 points:  1
i: 7 j: 10 points:  26
i: 7 j: 20 points:  46
i: 7 j: 30 points:  53
i: 7 j: 40 points:  62
i: 7 j: 50 points:  68
i: 7 j: 60 points:  82
i: 7 j: 70 points:  81
i: 7 j: 80 points:  89
i: 7 j: 90 points:  93
i: 7 j: 100 points:  94
i: 7 j: 110 points:  93
i: 8 j: 0 points:  1
i: 9 j: 0 points:  1
i: 10 j: 0 points:  1
i: 11 j: 0 points:  1
i: 12 j: 0 points:  1
i: 13 j: 0 points:  1
i: 14 j: 0 points:  1
i: 15 j: 0 points:  1
i: 16 j: 0 points:  1
i: 17 j: 0 points:  1
i: 18 j: 0 points:  1
i: 19 j: 0 points:  1
i: 20 j: 0 points:  1
i: 21 j: 0 points:  1
i: 22 j: 0 points:  1
i: 23 j: 0 points:  1
i: 24 j: 0 points:  1
i: 25 j: 0 points:  1
i: 26 j: 0 points:  1
i: 27 j: 0 points:  1
i: 28 j: 0 points:  1
i: 29 j: 0 points:  1
i: 30 j: 0 points:  1
i: 31 j: 0 points:  1
i: 32 j: 0 points:  1
i: 33 j:

i: 1228 j: 0 points:  1
i: 1229 j: 0 points:  1
i: 1230 j: 0 points:  1
i: 1231 j: 0 points:  1
i: 1232 j: 0 points:  1
i: 1233 j: 0 points:  1
i: 1234 j: 0 points:  1
i: 1235 j: 0 points:  1
i: 1236 j: 0 points:  1
i: 1237 j: 0 points:  1
i: 1238 j: 0 points:  1
i: 1239 j: 0 points:  1
i: 1240 j: 0 points:  1
i: 1241 j: 0 points:  1
i: 1242 j: 0 points:  1
i: 1243 j: 0 points:  1
i: 1244 j: 0 points:  1
i: 1245 j: 0 points:  1
i: 1246 j: 0 points:  1
i: 1247 j: 0 points:  1
i: 1248 j: 0 points:  1
i: 1249 j: 0 points:  1
i: 1250 j: 0 points:  1
i: 1251 j: 0 points:  1
i: 1252 j: 0 points:  1
i: 1253 j: 0 points:  1
i: 1254 j: 0 points:  1
i: 1255 j: 0 points:  1
i: 1256 j: 0 points:  1
i: 1257 j: 0 points:  1
i: 1258 j: 0 points:  1
i: 1259 j: 0 points:  1
i: 1260 j: 0 points:  1
i: 1261 j: 0 points:  1
i: 1262 j: 0 points:  1
i: 1263 j: 0 points:  1
i: 1264 j: 0 points:  1
i: 1265 j: 0 points:  1
i: 1266 j: 0 points:  1
i: 1267 j: 0 points:  1
i: 1268 j: 0 points:  1
i: 1269 j: 0 poi

i: 2060 j: 0 points:  1
i: 2061 j: 0 points:  1
i: 2062 j: 0 points:  1
i: 2063 j: 0 points:  1
i: 2064 j: 0 points:  1
i: 2064 j: 10 points:  19
i: 2065 j: 0 points:  1
i: 2065 j: 10 points:  24
i: 2066 j: 0 points:  1
i: 2066 j: 10 points:  11
i: 2067 j: 0 points:  1
i: 2067 j: 10 points:  28
i: 2068 j: 0 points:  1
i: 2068 j: 10 points:  14
i: 2069 j: 0 points:  1
i: 2069 j: 10 points:  28
i: 2070 j: 0 points:  1
i: 2071 j: 0 points:  1
i: 2072 j: 0 points:  1
i: 2073 j: 0 points:  1
i: 2073 j: 10 points:  22
i: 2074 j: 0 points:  1
i: 2075 j: 0 points:  1
i: 2075 j: 10 points:  25
i: 2076 j: 0 points:  1
i: 2077 j: 0 points:  1
i: 2078 j: 0 points:  1
i: 2079 j: 0 points:  1
i: 2079 j: 10 points:  22
i: 2079 j: 20 points:  25
i: 2080 j: 0 points:  1
i: 2080 j: 10 points:  12
i: 2081 j: 0 points:  1
i: 2082 j: 0 points:  1
i: 2083 j: 0 points:  1
i: 2084 j: 0 points:  1
i: 2085 j: 0 points:  1
i: 2085 j: 10 points:  14
i: 2085 j: 20 points:  17
i: 2086 j: 0 points:  1
i: 2087 j: 0 p

i: 2171 j: 320 points:  196
i: 2171 j: 330 points:  204
i: 2171 j: 340 points:  207
i: 2171 j: 350 points:  210
i: 2171 j: 360 points:  210
i: 2171 j: 370 points:  222
i: 2171 j: 380 points:  222
i: 2171 j: 390 points:  218
i: 2171 j: 400 points:  220
i: 2171 j: 410 points:  221
i: 2171 j: 420 points:  220
i: 2171 j: 430 points:  231
i: 2171 j: 440 points:  239
i: 2171 j: 450 points:  244
i: 2171 j: 460 points:  246
i: 2171 j: 470 points:  250
i: 2171 j: 480 points:  252
i: 2171 j: 490 points:  253
i: 2171 j: 500 points:  257
i: 2171 j: 510 points:  256
i: 2171 j: 520 points:  258
i: 2171 j: 530 points:  268
i: 2171 j: 540 points:  265
i: 2171 j: 550 points:  267
i: 2171 j: 560 points:  267
i: 2171 j: 570 points:  269
i: 2171 j: 580 points:  270
i: 2171 j: 590 points:  267
i: 2171 j: 600 points:  265
i: 2171 j: 610 points:  264
i: 2171 j: 620 points:  263
i: 2171 j: 630 points:  258
i: 2171 j: 640 points:  253
i: 2171 j: 650 points:  261
i: 2171 j: 660 points:  269
i: 2171 j: 670 point

i: 2210 j: 140 points:  180
i: 2210 j: 150 points:  177
i: 2211 j: 0 points:  1
i: 2211 j: 10 points:  74
i: 2211 j: 20 points:  92
i: 2211 j: 30 points:  119
i: 2211 j: 40 points:  131
i: 2211 j: 50 points:  135
i: 2211 j: 60 points:  145
i: 2211 j: 70 points:  157
i: 2211 j: 80 points:  166
i: 2211 j: 90 points:  171
i: 2211 j: 100 points:  171
i: 2211 j: 110 points:  186
i: 2211 j: 120 points:  187
i: 2211 j: 130 points:  199
i: 2211 j: 140 points:  209
i: 2211 j: 150 points:  210
i: 2212 j: 0 points:  1
i: 2212 j: 10 points:  74
i: 2212 j: 20 points:  92
i: 2212 j: 30 points:  121
i: 2212 j: 40 points:  129
i: 2212 j: 50 points:  137
i: 2212 j: 60 points:  145
i: 2212 j: 70 points:  153
i: 2212 j: 80 points:  162
i: 2212 j: 90 points:  168
i: 2212 j: 100 points:  168
i: 2212 j: 110 points:  186
i: 2212 j: 120 points:  183
i: 2212 j: 130 points:  200
i: 2212 j: 140 points:  200
i: 2212 j: 150 points:  202
i: 2212 j: 160 points:  200
i: 2212 j: 170 points:  196
i: 2213 j: 0 points:  

...

i: 2229 j: 900 points:  464
i: 2229 j: 910 points:  463
i: 2229 j: 920 points:  474
i: 2229 j: 930 points:  482
i: 2229 j: 940 points:  486
i: 2229 j: 950 points:  489
i: 2229 j: 960 points:  489
i: 2229 j: 970 points:  491
i: 2229 j: 980 points:  492
i: 2229 j: 990 points:  491
i: 2229 j: 1000 points:  490
i: 2229 j: 1010 points:  490
i: 2229 j: 1020 points:  492
i: 2229 j: 1030 points:  489
i: 2229 j: 1040 points:  484
i: 2229 j: 1050 points:  479
i: 2229 j: 1060 points:  469
i: 2229 j: 1070 points:  462
i: 2229 j: 1080 points:  452
i: 2229 j: 1090 points:  449
i: 2229 j: 1100 points:  446
i: 2229 j: 1110 points:  443
i: 2229 j: 1120 points:  440
i: 2229 j: 1130 points:  447
i: 2229 j: 1140 points:  447
i: 2229 j: 1150 points:  440
i: 2229 j: 1160 points:  434
i: 2229 j: 1170 points:  424
i: 2229 j: 1180 points:  417
i: 2229 j: 1190 points:  410
i: 2229 j: 1200 points:  405
i: 2229 j: 1210 points:  415
i: 2229 j: 1220 points:  415
i: 2229 j: 1230 points:  412
i: 2229 j: 1240 points: 

i: 2231 j: 950 points:  536
i: 2231 j: 960 points:  539
i: 2231 j: 970 points:  542
i: 2231 j: 980 points:  544
i: 2231 j: 990 points:  546
i: 2231 j: 1000 points:  548
i: 2231 j: 1010 points:  550
i: 2231 j: 1020 points:  547
i: 2231 j: 1030 points:  542
i: 2231 j: 1040 points:  547
i: 2231 j: 1050 points:  552
i: 2231 j: 1060 points:  554
i: 2231 j: 1070 points:  556
i: 2231 j: 1080 points:  556
i: 2231 j: 1090 points:  558
i: 2231 j: 1100 points:  557
i: 2231 j: 1110 points:  551
i: 2231 j: 1120 points:  549
i: 2231 j: 1130 points:  549
i: 2231 j: 1140 points:  552
i: 2231 j: 1150 points:  553
i: 2231 j: 1160 points:  550
i: 2231 j: 1170 points:  559
i: 2231 j: 1180 points:  565
i: 2231 j: 1190 points:  569
i: 2231 j: 1200 points:  573
i: 2231 j: 1210 points:  569
i: 2231 j: 1220 points:  565
i: 2231 j: 1230 points:  562
i: 2231 j: 1240 points:  552
i: 2231 j: 1250 points:  548
i: 2231 j: 1260 points:  544
i: 2231 j: 1270 points:  551
i: 2231 j: 1280 points:  560
i: 2231 j: 1290 poi

i: 2233 j: 1130 points:  667
i: 2233 j: 1140 points:  664
i: 2233 j: 1150 points:  664
i: 2233 j: 1160 points:  668
i: 2233 j: 1170 points:  667
i: 2233 j: 1180 points:  667
i: 2233 j: 1190 points:  660
i: 2233 j: 1200 points:  670
i: 2233 j: 1210 points:  673
i: 2233 j: 1220 points:  674
i: 2233 j: 1230 points:  675
i: 2233 j: 1240 points:  670
i: 2233 j: 1250 points:  672
i: 2233 j: 1260 points:  671
i: 2233 j: 1270 points:  666
i: 2233 j: 1280 points:  668
i: 2233 j: 1290 points:  670
i: 2233 j: 1300 points:  663
i: 2233 j: 1310 points:  666
i: 2233 j: 1320 points:  673
i: 2233 j: 1330 points:  667
i: 2233 j: 1340 points:  663
i: 2233 j: 1350 points:  666
i: 2233 j: 1360 points:  669
i: 2233 j: 1370 points:  660
i: 2233 j: 1380 points:  656
i: 2233 j: 1390 points:  652
i: 2233 j: 1400 points:  654
i: 2233 j: 1410 points:  658
i: 2233 j: 1420 points:  656
i: 2233 j: 1430 points:  648
i: 2233 j: 1440 points:  642
i: 2233 j: 1450 points:  637
i: 2233 j: 1460 points:  633
i: 2233 j: 147

i: 2235 j: 1060 points:  582
i: 2235 j: 1070 points:  584
i: 2235 j: 1080 points:  585
i: 2235 j: 1090 points:  599
i: 2235 j: 1100 points:  605
i: 2235 j: 1110 points:  609
i: 2235 j: 1120 points:  613
i: 2235 j: 1130 points:  617
i: 2235 j: 1140 points:  619
i: 2235 j: 1150 points:  620
i: 2235 j: 1160 points:  617
i: 2235 j: 1170 points:  617
i: 2235 j: 1180 points:  619
i: 2235 j: 1190 points:  614
i: 2235 j: 1200 points:  609
i: 2235 j: 1210 points:  615
i: 2235 j: 1220 points:  623
i: 2235 j: 1230 points:  629
i: 2235 j: 1240 points:  631
i: 2235 j: 1250 points:  637
i: 2235 j: 1260 points:  641
i: 2235 j: 1270 points:  641
i: 2235 j: 1280 points:  640
i: 2235 j: 1290 points:  642
i: 2236 j: 0 points:  1
i: 2236 j: 10 points:  74
i: 2236 j: 20 points:  92
i: 2236 j: 30 points:  124
i: 2236 j: 40 points:  136
i: 2236 j: 50 points:  144
i: 2236 j: 60 points:  160
i: 2236 j: 70 points:  174
i: 2236 j: 80 points:  184
i: 2236 j: 90 points:  190
i: 2236 j: 100 points:  194
i: 2236 j: 

i: 2236 j: 2710 points:  794
i: 2236 j: 2720 points:  784
i: 2236 j: 2730 points:  774
i: 2236 j: 2740 points:  771
i: 2236 j: 2750 points:  768
i: 2236 j: 2760 points:  766
i: 2236 j: 2770 points:  765
i: 2236 j: 2780 points:  765
i: 2236 j: 2790 points:  765
i: 2236 j: 2800 points:  765
i: 2236 j: 2810 points:  764
i: 2236 j: 2820 points:  765
i: 2236 j: 2830 points:  768
i: 2236 j: 2840 points:  773
i: 2236 j: 2850 points:  775
i: 2236 j: 2860 points:  779
i: 2236 j: 2870 points:  774
i: 2236 j: 2880 points:  780
i: 2236 j: 2890 points:  780
i: 2236 j: 2900 points:  784
i: 2236 j: 2910 points:  784
i: 2236 j: 2920 points:  777
i: 2236 j: 2930 points:  776
i: 2236 j: 2940 points:  779
i: 2236 j: 2950 points:  779
i: 2236 j: 2960 points:  782
i: 2236 j: 2970 points:  782
i: 2236 j: 2980 points:  778
i: 2236 j: 2990 points:  768
i: 2236 j: 3000 points:  766
i: 2236 j: 3010 points:  764
i: 2236 j: 3020 points:  767
i: 2236 j: 3030 points:  763
i: 2236 j: 3040 points:  767
i: 2236 j: 305

In [216]:
%%time
for r in [3, 5, 7, 10, 15, 19]:
    vessel_graph_cn = connect_endings_3d(vessel_graph, edt_img, r)
    print(nx.info(vessel_graph_cn))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Alicja\anaconda3\envs\im\lib\site-packages\IPython\core\magics\execution.py", line 1325, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 2, in <module>
  File "C:\Users\Alicja\AppData\Local\Temp\ipykernel_18408\1039737252.py", line -1, in connect_endings_3d
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Alicja\anaconda3\envs\im\lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\Alicja\anaconda3\envs\im\lib\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\Alicja\anaconda3\envs\im\lib\site-packages\IPython\core\ultratb.py", line 1199, in structured_traceback
    return VerboseTB.structured_traceback(
  File "C:\Users\Alicja\anaconda3\envs\im\li

In [264]:
model_with_graph = draw_graph_on_model(bin_reco, vessel_graph_cn)
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()

In [241]:
%%time
data_graph = parametrize_graph(vessel_graph_cn, edt_img)

CPU times: total: 0 ns
Wall time: 2.99 ms


C:\Users\Alicja\AppData\Local\Temp\ipykernel_18408\1430834244.py:398: RuntimeWarning: invalid value encountered in divide
  v1 /= np.linalg.norm(v1)
C:\Users\Alicja\AppData\Local\Temp\ipykernel_18408\1430834244.py:400: RuntimeWarning: invalid value encountered in divide
  v2 /= np.linalg.norm(v2)


In [242]:
%%time
data_graph_cl = clean_data_graph(data_graph)

IndexError: list index out of range

In [210]:
model_with_graph = draw_graph_on_model(bin_reco, data_graph_cl)

In [211]:
ColorMapVisualizer(model_with_graph.astype(np.uint8)).visualize()